In [ ]:
import numpy as np
import pandas as pd
import collections
from collections import Counter
from itertools import repeat, chain 
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import PorterStemmer
stemming = PorterStemmer()
from nltk.corpus import stopwords
stopsEnglish = set(stopwords.words("english"))
stopsGerman = set(stopwords.words("german")) 
stopsItalian = set(stopwords.words("italian")) 
stopsFrench = set(stopwords.words("french")) 
stopsRussian = set(stopwords.words("russian")) 
stopsSpanish = set(stopwords.words("spanish")) 

#Diavasma twn csv kai dimiourgia Dataframe me to Pandas.

RatingsDF = pd.read_csv('BX-Book-Ratings.csv',encoding= 'unicode_escape',delimiter = ';')
BooksDF = pd.read_csv('BX-Books.csv' ,encoding = 'latin-1', delimiter = ';')
UsersDF = pd.read_csv('BX-Users.csv' ,encoding = 'latin-1', delimiter = ';')

#Epeidi parakatw dimiourgeite provlima me to onoma User-ID(den dexetai to - ) to allazw.
RatingsDF = RatingsDF.rename(columns={'User-ID': 'UserID'})
UsersDF = UsersDF.rename(columns={'User-ID': 'UserID'})

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Teo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Teo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


G:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (3,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Preprocess

#Gia na svisoyme apo ta ratings kritikes vivliwn poy den uparxoyn sto BX-Books.csv.
RatingsDF = RatingsDF[RatingsDF.ISBN.isin(BooksDF.ISBN)]

#Kratame vivlia pou exoyn panw apo 10 kritikes
newBookRatings = RatingsDF[RatingsDF.groupby('ISBN')['ISBN'].transform('count').ge(10)]

#Gia na kratisoume sto RatingsDF mono tous Users pou vathmologisan panw apo 5 vivlia.
finalRatingsDF = newBookRatings[newBookRatings.groupby('UserID')['UserID'].transform('count').ge(5)]

#Gia na afairesoyme ta peritta books apo to BooksDF.
newBooksDF =(BooksDF[BooksDF.ISBN.isin(finalRatingsDF.ISBN)])


#Gia na afairesoyme toys perittous users apo to UsersDF.
newUsersDF =(UsersDF[UsersDF.UserID.isin(finalRatingsDF.UserID)])

#Metatropi titlwn se String:

newBooksDF['Keywords-Book-Title'] = newBooksDF['Book-Title'].astype(str)

#Metatropi se LowerCase:

newBooksDF['Keywords-Book-Title'] = newBooksDF['Keywords-Book-Title'].str.lower()

#Tokenization:

def identify_tokens(row):
    titles = row['Keywords-Book-Title']
    tokens = nltk.word_tokenize(titles)
    #Gia na kratisei mono tis lekseis kai oxi xaraktires kai noumera.
    token_words = [w for w in tokens if w.isalpha()]
    return token_words


newBooksDF['Keywords-Book-Title'] = newBooksDF.apply(identify_tokens, axis=1)


#Stemming:

def stem_list(row):
    my_list = row['Keywords-Book-Title']
    stemmed_list = [stemming.stem(title) for title in my_list]
    return (stemmed_list)

newBooksDF['Keywords-Book-Title'] = newBooksDF.apply(stem_list, axis=1)

#Remove Stopwords(English,German,Italian,French,Russian,Spanish):

newBooksDF['Keywords-Book-Title'] = newBooksDF['Keywords-Book-Title'].apply(lambda x: [item for item in x if item not in stopsEnglish])
newBooksDF['Keywords-Book-Title'] = newBooksDF['Keywords-Book-Title'].apply(lambda x: [item for item in x if item not in stopsGerman])
newBooksDF['Keywords-Book-Title'] = newBooksDF['Keywords-Book-Title'].apply(lambda x: [item for item in x if item not in stopsItalian])
newBooksDF['Keywords-Book-Title'] = newBooksDF['Keywords-Book-Title'].apply(lambda x: [item for item in x if item not in stopsFrench])
newBooksDF['Keywords-Book-Title'] = newBooksDF['Keywords-Book-Title'].apply(lambda x: [item for item in x if item not in stopsRussian])
newBooksDF['Keywords-Book-Title'] = newBooksDF['Keywords-Book-Title'].apply(lambda x: [item for item in x if item not in stopsSpanish])

#Ypologismos Dice Coefficient gia tis duo listes

def Dice_coefficient(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return (2*len(s1.intersection(s2))) / (len(s1) + len(s2))

#Ypologismos Jaccard Similarity gia tis duo listes

def jaccard_similarity(list1,list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

#Elegxos gia isotita anamesa stous suggrafeis

def writer_similarity(writerList,writer):
         if writer in writerList:
            return 1;
         else:
            return 0; 

#Ypologismos mikroteris diaforas apo eti ekdosis

def year_similarity(year_list,year):
    mindif = 1
    for i in year_list:
        difference = 1-(abs(int(i)-int(year))/2005)
        if (difference < mindif):
            mindif = difference
    return mindif

#Dimiourgia enos profil gia to xristi kai dataFrame me ta vivlia poy protima.

def user_preferences(user):
    dfUserScores = finalRatingsDF.loc[finalRatingsDF['UserID'] == user]
    dfPrefs = dfUserScores.nlargest(3,'Book-Rating')
    
    dfUserBooks =(newBooksDF[newBooksDF.ISBN.isin(dfPrefs.ISBN)])
    
    list_of_tag_list = dfUserBooks['Keywords-Book-Title'].tolist()
    
    
    #Uparxei lista mesa se lista ara me ton parakatw kwdika metatrepoyme ta tag se mia flat lista.
    
    tag_list = []
    for x in list_of_tag_list:
        for y in x:
            tag_list.append(y)
            
    writer_list = dfUserBooks['Book-Author'].tolist()
    year_list = dfUserBooks['Year-Of-Publication'].tolist()
    
    return dfUserScores, dfUserBooks, tag_list, writer_list, year_list


#Ypologismos tis omoiotitas me kathe antikeimeno toy Dataframe kai dimiourgia kainourgias stilis
#gia sto Dataframe gia Jaccard kai Dice.

def similarity_computation(userid,tag_list,writer_list,year_list):
    for index, row in booksToSuggest.iterrows():
        thisWriter = row['Book-Author']
        thisYear = row['Year-Of-Publication']
        thisTagList = row ['Keywords-Book-Title']
        jaccardSim = jaccard_similarity(tag_list,thisTagList)
        
        diceCoefficient = Dice_coefficient(tag_list,thisTagList)
        
        writerSim = writer_similarity(writer_list,thisWriter)
        
        yearSim = year_similarity(year_list,thisYear)
        
        rowSimilarityJaccard = (jaccardSim*0.2) + (writerSim*0.4) + (yearSim*0.4)
        rowSimilarityDice =  (diceCoefficient*0.5) + (writerSim*0.3) + (yearSim*0.2)
        booksToSuggest.at[index,'Jac-Similarity'] = rowSimilarityJaccard
        booksToSuggest.at[index,'Dice-Coefficient'] = rowSimilarityDice
    return booksToSuggest


#Ypologismos epikalipsis 2 listwn

def Overlap (list1,list2):
    s1 = set(list1)
    s2 = set(list2)
    
    return  len(s1.intersection(s2)) / min(len(s1),len(s2))


#Experiment No1:

#Tuxaia epilogi 5 users apo to Dataframe.
    
randUsers = newUsersDF['UserID'].sample(n=5).tolist()

for x in randUsers:
    
    dfUserScores, dfUserBooks, tag_list, writer_list, year_list = user_preferences(x)
    
    #Dimiourgia Dataframe me ta vivlia poy den exei vathmologisei o xristis
    booksToSuggest = newBooksDF[~newBooksDF.ISBN.isin(dfUserBooks.ISBN)]
   
    
    #klisi sinartisis upologismou omoiotitas
    similarity_computation(x,tag_list,writer_list,year_list)    
    
    #Fetch twn 10 pio omoiwn vivliwn me basi tin kathe enallaktiki kai export se csv arxeio.
    booksToSuggestJaccard = booksToSuggest.nlargest(10,'Jac-Similarity')
    booksToSuggestDice = booksToSuggest.nlargest(10,'Dice-Coefficient')
    booksToSuggestJaccard.to_csv('Jaccard_User%dRecommendations.csv' %(x), index = None, header=True,sep = ';')
    booksToSuggestDice.to_csv('Dice_User%dRecommendations.csv' %(x), index = None, header=True,sep = ';')
    
    
    


G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [5]:
# Experiment No2:

#Gia ton upologismo tou overlap kratw mia lista
OverlapList = []

for x in randUsers:
    
    overlapSum = 0
    
    #Fetch ta csv pou dimiourgithikan gia kathe xristi
    thisUserDfJaccard = pd.read_csv('Jaccard_User%dRecommendations.csv' %(x) ,encoding = 'latin-1', delimiter = ';')
    thisUserDfDice = pd.read_csv('Dice_User%dRecommendations.csv' %(x) ,encoding = 'latin-1', delimiter = ';')
    
    #Dimiourgia listwn me ta apotelesmata toy kathe xristi gia Jaccard kai Dice
    thisUserJaccardList = thisUserDfJaccard['ISBN'].tolist()
    thisUserDiceList = thisUserDfDice['ISBN'].tolist()
    
    #Ypologismos tis mesis epikalipsis kai append sti lista twn Overlap olwn twn xristwn.
    
    for j in range(1,11):
        overlapSum += Overlap( thisUserJaccardList[:j], thisUserDiceList[:j])
        
    thisUserOverlap = overlapSum/10
    OverlapList.append(thisUserOverlap)
print (OverlapList)

[1.0, 0.9309920634920635, 1.0, 1.0, 0.9888888888888889]


In [28]:
#Experiment No3:

OverlapListJac = []
OverlapListDice = []

for x in randUsers:
    UserISBNListofLists = []
    overlapSumJac = 0
    overlapSumDice = 0
    
    #Fetch ta csv pou dimiourgithikan gia kathe xristi
    thisUserDfJaccard = pd.read_csv('Jaccard_User%dRecommendations.csv' %(x) ,encoding = 'latin-1', delimiter = ';')
    thisUserDfDice = pd.read_csv('Dice_User%dRecommendations.csv' %(x) ,encoding = 'latin-1', delimiter = ';')
    
    #Dimiourgia listas me ta apotelesmata toy kathe xristi
    thisUserListJac = thisUserDfJaccard['ISBN'].tolist()
    thisUserListDice = thisUserDfDice['ISBN'].tolist()
    
    #Enwsi twn dio listwn
    thisUserList =  thisUserListJac + thisUserListDice
    
    #Taksinomisi tis Listas analoga me tis emfaniseis tou kathe vivlioy.
    thisUserSortedList = list(chain.from_iterable(repeat(i, c) 
         for i, c in Counter(thisUserList).most_common())) 
    
    #Dimiourgia tou 'Golden Standard' gia ton xristi me afairesi twn duplicate stoixeiwn
    goldenStandard = list(dict.fromkeys(thisUserSortedList))
    
    #Ypologismos epikalipsis tis kathe listas me ta 10 prwta stoixeia toy golden standard
    for j in range(1,11):
        overlapSumJac += Overlap( thisUserListJac[:j], goldenStandard[:j])
        overlapSumDice += Overlap( thisUserListDice[:j], goldenStandard[:j])
    
    thisUserOverlapJac = overlapSumJac/10
    thisUserOverlapDice = overlapSumDice/10
    OverlapListJac.append(thisUserOverlapJac)
    OverlapListDice.append(thisUserOverlapDice)
    
print (OverlapListJac,OverlapListDice)
            


[1.0, 1.0, 1.0, 1.0, 1.0] [1.0, 0.9309920634920635, 1.0, 1.0, 0.9888888888888889]
